In [6]:
# !pip3 install minio

from pyspark.sql import SparkSession
import requests
import json
import configparser

In [7]:
config = configparser.ConfigParser()
config.read('./airflow//config/config.ini')

['./airflow//config/config.ini']

In [11]:
# URL para download do arquivo
municipios_ibge_url = config['URL']['municipios_ibge']

# Defina as informações de conexão com o MinIO
minio_access_key = config['MinIO']['access_key']
minio_secret_key = config['MinIO']['secret_key']
minio_endpoint = config['MinIO']['endpoint']

# Bucket onde será armazenado o arquivo zip baixado da internet
minio_bucket = config['Bucket']['bucket_raw']
prefix = config['Bucket']['prefix_municipios_ibge_json']

# Nome do arquivo que será gravado no MinIO
municipios_ibge = config['FILE']['municipios_ibge_json']

In [9]:
# Inicializar a sessão Spark
spark = (SparkSession.builder
         .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint)
         .config("spark.hadoop.fs.s3a.access.key", minio_access_key)
         .config("spark.hadoop.fs.s3a.secret.key", minio_secret_key)
         .config("spark.hadoop.fs.s3a.path.style.access", "true")
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
         .getOrCreate()
         )

In [12]:
# Faz a requisição para obter os dados JSON
response = requests.get(municipios_ibge_url)
json_data = response.content.decode('utf-8')  # Decodifica usando UTF-8

In [13]:
# Transforma os dados JSON em uma string
json_string = json.dumps(json_data)

In [16]:
len(json_string)

3015762

In [25]:
# Define o caminho completo do diretório a ser excluído
# directory_to_delete = "s3a://" + minio_bucket + "/" + prefix + municipios_ibge
directory_to_delete = minio_bucket + "/" + prefix + municipios_ibge
print(directory_to_delete)

raw/municipios_ibge_json/MunicipiosIBGE.json


In [26]:
# Grava a string JSON no MinIO
rdd = spark.sparkContext.parallelize([json_string])
rdd.saveAsTextFile(directory_to_delete)

Py4JJavaError: An error occurred while calling o71.saveAsTextFile.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSy

In [21]:
rdd.values()

PythonRDD[1] at RDD at PythonRDD.scala:53